# 0) Versioning

Version for Medium article

# 1) Package Import

In [ ]:
# Importing
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import *
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.actions.wheel_input import ScrollOrigin
from selenium.webdriver.common.by import By

from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager


In [ ]:
import re as re
import time
import pandas as pd

In [ ]:
import random

# 2) Variable declartations (global)

In [ ]:
# webdriver options: tbc if necessary
options = Options()
options.add_argument("start-maximized")

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
USERNAME = "user.name@company.xyu"
PASSWORD = "1234_if_you_are_really_stupid"
print(USERNAME)
print(PASSWORD)

In [ ]:
# Main keyword for search:
###############################################################
# keywords4search = ['VP Finance']
# keywords4search = ['Director FP&A']
keywords4search = ['Director Finance']
###############################################################
# construct link from keyword input
s_search_pos = ''   # search string for position
for i in keywords4search:
    i2 = i.replace(' ', '%20')
    s_search_pos+=(i2)+'%20'
s_search_pos=s_search_pos[:-3]

In [ ]:
###########################################################################################################
# Further filter criteria for the search:
###########################################################################################################
s_pos_level = 'f_E=5%2C6'             # f_E: Filter Employment type? -> only Direktoren, VP oder GL-Level
s_freshness = 'f_TPR=r2592000'       # code for "position posted within last month"
s_geo_loc = 'geoId=91000002'         # 'geoId=104008204' = Düsseldorf  # 'geoId=101282230' = Deutschland # 'geoId=91000006' = DACH  # 'geoId=103480659' = NRW 
                                     # 'geoId=91000006' = Benelux # 'geoId=105072130' = Polen  # 'geoId=106693272' = Schweiz # 'geoId=100506914' = Europe
                                     # 'geoId=91000002'= Europäischer Wirtschaftsraum
# "Position"-filter
s_position = 'f_T=1088%2C4866%2C4924%2C3968%2C5%2C127%2C1208%2C68'

# 3) Definitions

## 3a) Class definitions

## 3b) Function Definitions

In [ ]:
def f_scrap_position_details():
    # Details to be scrapped from the individual clicked card aka job posting
    JobTitle = 'initial_value'
    JobTitleLink = 'initial_value'
    CompanyName = 'initial_value'
    CompanyLocation = 'initial_value'
    CompanySize = 'initial_value'
    PositionLevel = 'initial_value'
    JobDescDetails = 'initial_value'
    
    try:
     JobTitle = driver.find_element(By.CLASS_NAME,"jobs-unified-top-card__job-title").text
    except:
     JobTitle = "JobTitle not found"   
    try:
        JobTitleLink = driver.find_element(By.CLASS_NAME,"jobs-unified-top-card__content--two-pane").find_element(By.TAG_NAME,"a").get_attribute('href')
    except:
       JobTitle = "JobTitleLink not found"
    try:
        CompanyName = driver.find_element(By.CLASS_NAME,"jobs-unified-top-card__company-name").find_element(By.TAG_NAME,"a").text
    except:
        CompanyName = "CompanyName not found"
    try:
        CompanyLocation = driver.find_element(By.CLASS_NAME,"jobs-unified-top-card__bullet").text
    except:
        CompanyLocation = "CompanyLocation not found"
 
    JobDescElements = driver.find_element(By.CLASS_NAME,"jobs-description-content__text").find_elements(By.TAG_NAME,"span")
    for element in JobDescElements:
        try:
            if len(element.text) > 50:
                JobDescDetails = element.text
                break
            else:
                JobDescDetails = "These are not the JobDescDetails you were looking for"
        except:
            JobDescDetails = "JobDescDetails not found"
    
    try:
        CompanyInsights = driver.find_elements(By.CLASS_NAME,'.'.join("jobs-unified-top-card__job-insight".split()))
        PositionLevel = CompanyInsights[0].text
        CompanySize = CompanyInsights[1].text
    except:
        PositionLevel = "PostionLevel not found"
        CompanySize = "CompanySize not found"
    
    l_return = [JobTitle, JobTitleLink, CompanyName, CompanyLocation, CompanySize, PositionLevel, JobDescDetails]
    
    return (l_return)

# 4) Actual Program

## 4a) Set up connection to website with credentials

In [ ]:
# Use driver to open the link
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
driver.get("https://www.linkedin.com/uas/login")
time.sleep(4)

In [ ]:
# Use login credentials to login
email=driver.find_element(By.ID, "username")
email.send_keys(USERNAME)
password=driver.find_element(By.ID, "password")
password.send_keys(PASSWORD)
time.sleep(3)
password.send_keys(Keys.RETURN)

## 4b) Getting initial information on search

In [ ]:
######################################################################################################################################
# Complete search string -> ACTUAL POSITIONS TO BE SCRAPED ARE THE ANSWER TO THE CALL OF THIS URL
#####################################################################################################################################
s_searchURL = f'https://www.linkedin.com/jobs/search/?&{s_pos_level}&{s_position}&{s_freshness}&{s_geo_loc}&keywords={s_search_pos}'
print(s_searchURL)
#####################################################################################################################################

In [ ]:
driver.get(s_searchURL)

In [ ]:
result = []     
link = driver.current_url
driver.get(link)

In [ ]:
time.sleep(4)

In [ ]:
# initiatate l_links with nonsense values, but with the rights length to serve as check value in coming WHILE loop
#l_links = [1] * len(driver.find_elements(By.CSS_SELECTOR,'a.disabled.ember-view.job-card-container__link.job-card-list__title'))

In [ ]:
no_posts = int(driver.find_element(By.CSS_SELECTOR,'small.display-flex.t-12.t-black--light.t-normal.jobs-search-results-list__text').text.split(' ')[0])
print(f'For position {s_search_pos} there are {no_posts} vacancies.')

In [ ]:
npages = (no_posts//25)+1
print(f'No. of pages with results: {npages}')

## 4c) Main loop: loop over result pages, force "slow scrolling" thru all results and retrieve detail info from all cards

In [ ]:
result = []

In [ ]:
for i in range(0, npages*25, 25):
    print("Page:",int((i/25)+1),"of",npages)
    s_searchURL = link+"&start="+str(i)
    print(s_searchURL)
    driver.get(s_searchURL)
    time.sleep(4)
    
    
    # get the number of initially available job cards (e.g. usually only the first 7 are loaded)
    # but BREAK the for-loop (aka abandon) in case NO such card is found (this can happen if e.g. the
    # number of hits after loading several pages suddenly has become smaller - and as the algorithm
    # only remembers the higher, original number of hits (and resulting pages), it would try to load
    # a page that does not exist (or rather: that does show no search results) anymore
    try:
        l_links_short = [1] * len(driver.find_elements(By.CSS_SELECTOR,'a.disabled.ember-view.job-card-container__link.job-card-list__title'))
    except:
        break
    
    # find the FIRST job card
    origin_elem = driver.find_element(By.CSS_SELECTOR,'a.disabled.ember-view.job-card-container__link.job-card-list__title')
    v_while_cycles = 0

    # forced slow scrolling in order to force loading of all job cards
    while (len(l_links_short) < 25) and (v_while_cycles < 12):
        v_while_cycles += 1
        ###################################################################################################################
        # scroll 300 down from the first job card
        scroll_origin = ScrollOrigin.from_element(origin_elem, 0, 300)
        # and nw scroll 300 deeper down with every new cycle
        ActionChains(driver).scroll_from_origin(scroll_origin, 0, (v_while_cycles * 300)).perform() 
        ###################################################################################################################
        ######################
        time.sleep(2)
        ######################

        l_links_long = []
        l_links_short = []
        l_titles_short = []
        for l in driver.find_elements(By.CSS_SELECTOR,'a.disabled.ember-view.job-card-container__link.job-card-list__title'):
            txt_long = str(l.get_attribute('text'))
            title_short = txt_long[15:]
            title_short = title_short.split("\n")[0]
            l_titles_short.append(title_short)

            ref_long = str(l.get_attribute('href'))
            ref_short = ref_long.split("?")[0]
            l_links_long.append(ref_long)
            l_links_short.append(ref_short)

        print( f'CONTROL: new number of job links after scrolling {v_while_cycles}: value = {len(l_links_short)}')

    print('################################################')
    print (l_titles_short)
    print (len(driver.find_elements(By.CSS_SELECTOR,'a.disabled.ember-view.job-card-container__link.job-card-list__title')))

    # now we're sure to have all max. 25 job cards an need to iterate ofer them in order to click on every single
    # one and retrieve the details from the it
    for l in driver.find_elements(By.CSS_SELECTOR,'a.disabled.ember-view.job-card-container__link.job-card-list__title'):
        try:
            l.click()
        except:
            pass        
        time.sleep(random.randint(3,5))
        try:
            result.append(f_scrap_position_details())
        except:
            pass
           

## 4d) Bring result into data frame format and export data frame to (Excel) file

In [ ]:
df = pd.DataFrame(result, columns = ["JobTitle", "JobTitleLink", "CompanyName","CompanyLocation", "CompanySize", "PositionLevel","JobDescDetails"])

In [ ]:
# replace line return from html via regex
df['JobDescDetails'] = df['JobDescDetails'].replace(r'\s+|\\n', ' ', regex=True)

In [ ]:
# Split long link so as to keep only the essential part, the direct linkto the job post
df[['DirectJobLink','garbage']] = df['JobTitleLink'].str.split("?", n=1, expand=True)

In [ ]:
df = df.drop(['JobTitleLink', 'garbage'], axis = 1)

In [ ]:
df.head(5)

In [ ]:
#df.to_csv("_LINKEDIN_JOB_POSTINGS.xlsx")     # no openpyxl ????
df.to_excel("_LINKEDIN_JOB_POSTINGS.xlsx")